# ACDC Issuance with KLI: Issuer-To-Issuee

<div class="alert alert-primary">
<b>🎯 OBJECTIVE</b><hr>
Demonstrate how to issue chained Authentic Chained Data Containers (ACDCs) using an Issuer-To-Issuee (<code>I2I</code>) edge relationship with the KERI Command Line Interface (KLI). It also illustrates how to embed a simple rule within an ACDC. We will implement the "Endorsement for Building Access" scenario.
</div>

## Scenario Recap: Endorsement for Building Access

This notebook focuses on the practical KLI commands for implementing an `I2I` chained credential scenario. For a detailed theoretical explanation of ACDC Edges, Edge Operators, and Rules, please refer to the "Advanced ACDC Features: Edges, Edge Operators, and Rules" notebook. To summarize: 

- **ACME Corp** issues a "Role Credential" to an Employee.
- The **Employee**, by virtue of their "Role Credential", issues an "Access Credential" to a **Sub-contractor**.
- The **Access Credential** contains an `I2I` edge linking back to the Employee's "Role Credential", signifying that the Employee's authority to grant access is derived from their managerial role.
- The **Access Credential** will also include a simple textual rule regarding its non-transferability. 

## Initial Setup: Keystores, AIDs, Registries, and OOBIs
As usual, it is necessary to set up our participants:

- Acme Corporation (`acme_aid`): The initial authority in this scenario, responsible for issuing the Role Credential.
- Employee (`employee_aid`): This participant will first receive the Role Credential from Acme and subsequently issue the Access Credential.
- Sub-contractor (`subcontractor_aid`): The recipient of the Access Credential.

For each participant:
- Initialize their respective keystores.
- Incept their Autonomic Identifiers (AIDs). These AIDs will be configured as transferable and will utilize the default witness setup from `keystore_init_config.json`.
- Establish OOBI connections. This involves generating OOBIs for each AID and resolving them to ensure all necessary participants (Acme-Employee, Employee-Sub-contractor) can securely discover each other.

For ACMR and the Employee:
- Incept a credential registry

In [1]:
# Imports and Utility functions
from scripts.utils import exec, clear_keri, pr_title, pr_message, pr_continue
from scripts.saidify import get_schema_said
import json, os

clear_keri()

# ACME Keystore and AID
acme_keystore_name = "acme_ks"
acme_salt = exec("kli salt")
acme_aid_alias = "acme"
acme_registry_name = "acme_mgr_registry"

# Employee Keystore and AID
employee_keystore_name = "employee_ks"
employee_salt = exec("kli salt")
employee_aid_alias = "employee"
employee_registry_name = "employee_access_registry"

# Sub-contractor Keystore and AID
subcontractor_keystore_name = "subcontractor_ks"
subcontractor_salt = exec("kli salt")
subcontractor_aid_alias = "subcontractor"

pr_title("Initializing keystores")

!kli init --name {acme_keystore_name} \
    --nopasscode \
    --salt {acme_salt} \
    --config-dir ./config \
    --config-file keystore_init_config.json

!kli init --name {employee_keystore_name} \
    --nopasscode \
    --salt {employee_salt} \
    --config-dir ./config \
    --config-file keystore_init_config.json

!kli init --name {subcontractor_keystore_name} \
    --nopasscode \
    --salt {subcontractor_salt} \
    --config-dir ./config \
    --config-file keystore_init_config.json

pr_title("Initializing AIDs")

!kli incept --name {acme_keystore_name} \
    --alias {acme_aid_alias} \
    --file ./config/aid_inception_config.json # Uses witnesses and transferable settings

!kli incept --name {employee_keystore_name} \
    --alias {employee_aid_alias} \
    --file ./config/aid_inception_config.json

!kli incept --name {subcontractor_keystore_name} \
    --alias {subcontractor_aid_alias} \
    --file ./config/aid_inception_config.json

pr_title("Initializing Credential Registries")

!kli vc registry incept --name {acme_keystore_name} \
    --alias {acme_aid_alias} \
    --registry-name {acme_registry_name}

!kli vc registry incept --name {employee_keystore_name} \
    --alias {employee_aid_alias} \
    --registry-name {employee_registry_name}


acme_aid_prefix = exec(f"kli aid --name {acme_keystore_name} --alias {acme_aid_alias}")
employee_aid_prefix = exec(f"kli aid --name {employee_keystore_name}  --alias {employee_aid_alias}")
subcontractor_aid_prefix = exec(f"kli aid --name {subcontractor_keystore_name}  --alias {subcontractor_aid_alias}")

pr_message(f"ACME AID: {acme_aid_prefix}")
pr_message(f"Employee AID: {employee_aid_prefix}")
pr_message(f"Sub-contractor AID: {subcontractor_aid_prefix}")

pr_title("Generating and resolving OOBIs")

# ACME and Employee OOBI Exchange
acme_oobi = exec(f"kli oobi generate --name {acme_keystore_name} --alias {acme_aid_alias} --role witness")
employee_oobi = exec(f"kli oobi generate --name {employee_keystore_name} --alias {employee_aid_alias} --role witness")

!kli oobi resolve --name {acme_keystore_name} \
    --oobi-alias {employee_aid_alias} \
    --oobi {employee_oobi}

!kli oobi resolve --name {employee_keystore_name} \
    --oobi-alias {acme_aid_alias} \
    --oobi {acme_oobi}

# Employee and Sub-contractor OOBI Exchange
subcontractor_oobi = exec(f"kli oobi generate --name {subcontractor_keystore_name}  --alias {subcontractor_aid_alias} --role witness")

!kli oobi resolve --name {employee_keystore_name} \
    --oobi-alias {subcontractor_aid_alias} \
    --oobi {subcontractor_oobi}

!kli oobi resolve --name {subcontractor_keystore_name} \
    --oobi-alias {employee_aid_alias} \
    --oobi {employee_oobi}

pr_message("OOBI connections established.")

pr_continue()

Proceeding with deletion of '/usr/local/var/keri/' without confirmation.
✅ Successfully removed: /usr/local/var/keri/

  Initializing keystores  

KERI Keystore created at: /usr/local/var/keri/ks/acme_ks
KERI Database created at: /usr/local/var/keri/db/acme_ks
KERI Credential Store created at: /usr/local/var/keri/reg/acme_ks

Loading 3 OOBIs...
http://witness-demo:5642/oobi/BBilc4-L3tFUnfM_wJr4S4OJanAv_VmF_dJNN6vkf2Ha/controller?name=Wan&tag=witness succeeded
http://witness-demo:5643/oobi/BLskRTInXnMxWaGqcpSyMgo0nYbalW99cGZESrz3zapM/controller?name=Wes&tag=witness succeeded
http://witness-demo:5644/oobi/BIKKuvBwpmDVA4Ds-EpL5bt9OqPzWPja2LigFYZN2YfX/controller?name=Wil&tag=witness succeeded
KERI Keystore created at: /usr/local/var/keri/ks/employee_ks
KERI Database created at: /usr/local/var/keri/db/employee_ks
KERI Credential Store created at: /usr/local/var/keri/reg/employee_ks

Loading 3 OOBIs...
http://witness-demo:5642/oobi/BBilc4-L3tFUnfM_wJr4S4OJanAv_VmF_dJNN6vkf2Ha/controller?name

## Schema Definitions

We need two ACDC schemas:
- Role Schema (`role_schema.json`): For the credential ACME issues to the Employee.
- Access Schema (`access_schema.json`): For the credential the Employee issues to the Sub-contractor. This schema will include definitions for an `e` (edges) section to specify the I2I link and an `r` (rules) section.

<div class="alert alert-info">
<b>ℹ️ NOTE</b><hr>
For this notebook,the schemas have been SAIDified and made available on a schema server. The SAIDification process was covered in the "SAIDifying ACDC Schemas" notebook.

### Role Schema

This schema defines the structure of the "Role Credential." It has a structure that is rather similar to the other schemas presented so far during the training:

- Filename: `role_schema.json` (content shown SAIDified)

In [2]:
role_schema_path = "config/schemas/role_schema.json" 
pr_title(f"Schema: {role_schema_path}")

role_schema_said = get_schema_said(role_schema_path)
pr_message(f"Schema SAID: {role_schema_said}")

pr_message(f"Retieving Schema from Server:")
!curl -s http://vlei-server:7723/oobi/{role_schema_said} | jq

pr_continue()


  Schema: config/schemas/role_schema.json  


Schema SAID: ENWatfUaeryBqvGnG7VdILVcqk84_eoxmiaJYguJXaRw


Retieving Schema from Server:

{
  "$id": "ENWatfUaeryBqvGnG7VdILVcqk84_eoxmiaJYguJXaRw",
  "$schema": "http://json-schema.org/draft-07/schema#",
  "title": "RoleCredential",
  "description": "Credential signifying a role within an organization.",
  "type": "object",
  "credentialType": "RoleCredential",
  "version": "1.0.0",
  "properties": {
    "v": {
      "description": "Credential Version String",
      "type": "string"
    },
    "d": {
      "description": "Credential SAID",
      "type": "string"
    },
    "u": {
      "description": "One time use nonce",
      "type": "string"
    },
    "i": {
      "description": "Issuer AID",
      "type": "string"
    },
    "ri": {
      "description": "Registry SAID",
      "type": "string"
    },
    "s": {
      "description": "Schema SAID",
      "type": "string"
    },
    "a": {
      "oneOf": [
        {
          "descripti

### Access Schema
This schema defines the "Access Credential". It includes an `e` (edges) section for the `I2I` link to the Role Credential and an `r` (rules) section for a usage policy.

Filename: `access_schema.json` (content shown SAIDified)


In [3]:
access_schema_path = "config/schemas/access_schema.json" 
pr_title(f"Schema: {access_schema_path}")

access_schema_said = get_schema_said(access_schema_path)
pr_message(f"Schema SAID: {access_schema_said}")

pr_message(f"Retieving Schema from Server:")
!curl -s http://vlei-server:7723/oobi/{access_schema_said} | jq

pr_continue()


  Schema: config/schemas/access_schema.json  


Schema SAID: EF2zX3g5YDyHMSjgsK4OayZMFmLRMxcAJfW363JhBOfD


Retieving Schema from Server:

{
  "$id": "EF2zX3g5YDyHMSjgsK4OayZMFmLRMxcAJfW363JhBOfD",
  "$schema": "http://json-schema.org/draft-07/schema#",
  "title": "AccessCredential",
  "description": "Credential granting access to a specific building or area, endorsed by a manager.",
  "type": "object",
  "credentialType": "AccessCredential",
  "version": "1.0.0",
  "properties": {
    "v": {
      "description": "Credential Version String",
      "type": "string"
    },
    "d": {
      "description": "Credential SAID",
      "type": "string"
    },
    "u": {
      "description": "One time use nonce",
      "type": "string"
    },
    "i": {
      "description": "Issuer AID (Employee's AID)",
      "type": "string"
    },
    "ri": {
      "description": "Registry SAID",
      "type": "string"
    },
    "s": {
      "description": "Schema SAID",
      "type": "string"
    },
    "a

## Resolving Schema OOBIs
All parties need to resolve the OOBIs for these schemas from the schema server.

In [4]:
pr_title("Resolving schema OOBIs")

role_schema_oobi = f"http://vlei-server:7723/oobi/{role_schema_said}"
access_schema_oobi = f"http://vlei-server:7723/oobi/{access_schema_said}"

# ACME resolves both
!kli oobi resolve --name {acme_keystore_name} --oobi-alias "role_schema" --oobi {role_schema_oobi}
!kli oobi resolve --name {acme_keystore_name} --oobi-alias "access_schema" --oobi {access_schema_oobi}

# Employee resolves both
!kli oobi resolve --name {employee_keystore_name} --oobi-alias "role_schema" --oobi {role_schema_oobi}
!kli oobi resolve --name {employee_keystore_name} --oobi-alias "access_schema" --oobi {access_schema_oobi}

# Sub-contractor resolves access_schema 
# might also need role_schema if they were to fully validate the chain themselves, will keep it simple for the example
!kli oobi resolve --name {subcontractor_keystore_name} --oobi-alias "access_schema" --oobi {access_schema_oobi}

pr_message("Schema OOBIs resolved.")
pr_continue()


  Resolving schema OOBIs  

http://vlei-server:7723/oobi/ENWatfUaeryBqvGnG7VdILVcqk84_eoxmiaJYguJXaRw resolved
http://vlei-server:7723/oobi/EF2zX3g5YDyHMSjgsK4OayZMFmLRMxcAJfW363JhBOfD resolved
http://vlei-server:7723/oobi/ENWatfUaeryBqvGnG7VdILVcqk84_eoxmiaJYguJXaRw resolved
http://vlei-server:7723/oobi/EF2zX3g5YDyHMSjgsK4OayZMFmLRMxcAJfW363JhBOfD resolved
http://vlei-server:7723/oobi/EF2zX3g5YDyHMSjgsK4OayZMFmLRMxcAJfW363JhBOfD resolved

Schema OOBIs resolved.


  You can continue ✅  




## Issuing credentials

Now that the setup is complete and the schemas are available, its necessary to create the credential chain. 


### Step 1: Role Credential Issuance

The Keystores, AIDs, and Credential Registry for ACME Corp. were created in the initial setup. The next step is to make the credential.

**ACME Creates Role Credential Data**

Create a JSON file `role_cred_data.json` with the attributes for this specific credential

In [5]:
pr_title("Creating role credential data")

!echo '{ \
    "roleTitle": "Engineering Manager", \
    "department": "Technology Innovations" \
}' > config/credential_data/role_cred_data.json

!cat config/credential_data/role_cred_data.json | jq

pr_continue()


  Creating role credential data  

{
  "roleTitle": "Engineering Manager",
  "department": "Technology Innovations"
}

  You can continue ✅  




**ACME Issues Role Credential to Employee**

In [6]:
pr_title("Creating Role credential")

issue_time_acme = exec("kli time")
!kli vc create --name {acme_keystore_name} \
    --alias {acme_aid_alias} \
    --registry-name {acme_registry_name} \
    --schema {role_schema_said} \
    --recipient {employee_aid_prefix} \
    --data "@./config/credential_data/role_cred_data.json" \
    --time {issue_time_acme}

role_credential_said = exec(f"kli vc list --name {acme_keystore_name} --alias {acme_aid_alias} --issued --said --schema {role_schema_said}")
pr_message(f"Role Credential SAID: {role_credential_said}")

pr_continue()


  Creating Role credential  

Waiting for TEL event witness receipts
Sending TEL events to witnesses
EPaILPfyemALrAm7zBFTKF2ugJQIV-58RUQE0iuP8cgr has been created.

Role Credential SAID: EPaILPfyemALrAm7zBFTKF2ugJQIV-58RUQE0iuP8cgr


  You can continue ✅  




**IPEX Transfer: ACME Grants, Employee Admits Manager Credential**

In [7]:
pr_title("Transfering credential (ipex grant)")

time = exec("kli time")
!kli ipex grant --name {acme_keystore_name} \
    --alias {acme_aid_alias} \
    --said {role_credential_said} \
    --recipient {employee_aid_prefix} \
    --time {time}

pr_title("Admiting credential (ipex grant)")
# Employee polls for the grant and admits it
employee_grant_msg_said = exec(f"kli ipex list --name {employee_keystore_name} --alias {employee_aid_alias} --poll --said")
time = exec("kli time")
!kli ipex admit --name {employee_keystore_name} \
    --alias {employee_aid_alias} \
    --said {employee_grant_msg_said} \
    --time {time}

# Employee lists the received credential
pr_message("\nEmployee's received Manager Credential:")
!kli vc list --name {employee_keystore_name} --alias {employee_aid_alias} --verbose

pr_continue()


  Transfering credential (ipex grant)  

Sending message EJNdgiWrgJadGNM3AbcXitfQMyLD6Hcpk8vHzQKfbqKt to EJW60Q4ISu3ZgWXWrBJEG6LDiIMBTbXbcnwlx8lnAc52
... grant message sent

  Admiting credential (ipex grant)  

Sending admit message to EKBk6nqGgkR7J4QEgqXf7Uoo6RBiIThd0mgqlA1ZrzmI
... admit message sent


Employee's received Manager Credential:

Current received credentials for employee (EJW60Q4ISu3ZgWXWrBJEG6LDiIMBTbXbcnwlx8lnAc52):

Credential #1: EPaILPfyemALrAm7zBFTKF2ugJQIV-58RUQE0iuP8cgr
    Type: RoleCredential
    Status: Issued ✔
    Issued by EKBk6nqGgkR7J4QEgqXf7Uoo6RBiIThd0mgqlA1ZrzmI
    Issued on 2025-06-05T06:59:00.947621+00:00
    Full Credential:
	{
	  "v": "ACDC10JSON0001c2_",
	  "d": "EPaILPfyemALrAm7zBFTKF2ugJQIV-58RUQE0iuP8cgr",
	  "i": "EKBk6nqGgkR7J4QEgqXf7Uoo6RBiIThd0mgqlA1ZrzmI",
	  "ri": "EGLzLesZIjmMs4661d8sTT0Tpro9zKoqJXZHckwdF6PJ",
	  "s": "ENWatfUaeryBqvGnG7VdILVcqk84_eoxmiaJYguJXaRw",
	  "a": {
	    "d": "EPkv1SNoDuqQCr6USTihtAwzhEcTLDH8ro-Fx3qB5RLZ",
	 

### Step 2: Access Granted Credential with I2I Edge & Rule

The Employee, now holding the "Role Credential", issues the "Access Credential" to the Sub-contractor. This new credential will link to the Manager Credential via an `I2I` edge and include a "Usage Disclaimer" rule.

**Employee Defines Access Granted Credential Data (with Edge and Rule)**

Create a JSON file `access_cred_data.json`. This file will contain the attributes for the access pass, the edge (e) pointing to the Employee's `role_credential_said`, and the rule (`r`).

In [8]:
access_cred_data_file_path = "config/credential_data/access_cred_data.json"

access_data = {
    "buildingId": "HQ-EastWing", 
    "accessLevel": "Level 2 - Common Areas & Labs" 
}

with open(access_cred_data_file_path, 'w') as f:
    json.dump(access_data, f, indent=4)

!cat {access_cred_data_file_path} | jq


{
  "buildingId": "HQ-EastWing",
  "accessLevel": "Level 2 - Common Areas & Labs"
}


In [9]:
access_cred_edge_file_path = "config/credential_data/access_cred_edge.json"

access_edge = {
    "d": "",
    "manager_endorsement": {
        "n": employee_aid_prefix,
        "s": role_schema_said,
        "o": "I2I"
    }
}

with open(access_cred_edge_file_path, 'w') as f:
    json.dump(access_edge, f, indent=4)

!kli saidify --file {access_cred_edge_file_path}

!cat {access_cred_edge_file_path} | jq


{
  "d": "EGERvQWtEnQv9JVnQQv6ASLxXo0VcyL5So78Jry8B3hh",
  "manager_endorsement": {
    "n": "EJW60Q4ISu3ZgWXWrBJEG6LDiIMBTbXbcnwlx8lnAc52",
    "s": "ENWatfUaeryBqvGnG7VdILVcqk84_eoxmiaJYguJXaRw",
    "o": "I2I"
  }
}


In [10]:
access_cred_rule_file_path = "config/credential_data/access_cred_rule.json"

access_rule = {
  "d": "",
  "usageDisclaimer": {
    "l": "This mock credential grants no actual access. For illustrative use only."
  }
}

with open(access_cred_rule_file_path, 'w') as f:
    json.dump(access_rule, f, indent=4)

!kli saidify --file {access_cred_rule_file_path}

!cat {access_cred_rule_file_path} | jq


{
  "d": "EGVMk928-Fz4DK2NSvZgtG0JJrMlrpxvuxBKPvFxfPSQ",
  "usageDisclaimer": {
    "l": "This mock credential grants no actual access. For illustrative use only."
  }
}


In [13]:
time = exec("kli time")
!kli vc create --name {employee_keystore_name} \
    --alias {employee_aid_alias} \
    --registry-name {employee_registry_name} \
    --schema {access_schema_said} \
    --recipient {subcontractor_aid_alias} \
    --data  "@./config/credential_data/access_cred_data.json" \
    --edges "@./config/credential_data/access_cred_edge.json" \
    --rules "@./config/credential_data/access_cred_rule.json" \
    --time {time}



Waiting for TEL event witness receipts
Sending TEL events to witnesses
^C


<div class="alert alert-primary">
  <b>📝 SUMMARY</b><hr>
  
</div>

In [15]:
access_schema_said

'EF2zX3g5YDyHMSjgsK4OayZMFmLRMxcAJfW363JhBOfD'

In [14]:
!kli vc create -h

usage: kli vc create [-h] --name NAME [--registry-name REGISTRY_NAME]
                     [--schema SCHEMA] [--edges EDGES] [--rules RULES]
                     [--recipient RECIPIENT] [--data DATA]
                     [--credential CREDENTIAL] [--base BASE] --alias ALIAS
                     [--private]
                     [--private-credential-nonce PRIVATE_CREDENTIAL_NONCE]
                     [--private-subject-nonce PRIVATE_SUBJECT_NONCE]
                     [--passcode BRAN] [--time TIME]

Initialize a prefix

options:
  -h, --help            show this help message and exit
  --name NAME, -n NAME  Human readable reference
  --registry-name REGISTRY_NAME, -r REGISTRY_NAME
                        Human readable name for registry, defaults to name of
                        Habitat
  --schema SCHEMA, -s SCHEMA
                        qb64 SAID of Schema to issue
  --edges EDGES, -e EDGES
                        AC/DC Edge links
  --rules RULES         AC/DC Rules Section
  --re

In [ ]:
from scripts.saidify import process_schema_file

# def process_schema_file(input_filepath: str, output_filepath: str, indent_output: bool = True):
process_schema_file("config/schemas/role_schema.bak.json", "config/schemas/role_schema.json")

process_schema_file("config/schemas/access_schema.bak.json", "config/schemas/access_schema.json")
